# Exploration de l'API Gmail

In [1]:
import os
import pickle
import base64
from email.mime.text import MIMEText
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

## Authentification Gmail (OAuth2)

In [5]:
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def gmail_authenticate():
    creds = None
    if os.path.exists('../token.pickle'):
        with open('../token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Attention au port utilisé ici :
            flow = InstalledAppFlow.from_client_secrets_file('../credentials.json', SCOPES)
            creds = flow.run_local_server(port=8083)
        with open('../token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)


## Envoi d’un e-mail

In [6]:
def send_email(subject, body, to_email):
    service = gmail_authenticate()
    message = MIMEText(body)
    message['to'] = to_email
    message['subject'] = subject
    raw_message = {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}
    message = service.users().messages().send(userId="me", body=raw_message).execute()
    print(f"E-mail envoyé ✅ ID: {message['id']}")


In [ ]:
# Remplacez "ton_mail" par votre vrai mail
# Je ne laisse pas mon mail ici pour des raisons de sécurité
send_email("Test Gmail API", "Ceci est un test depuis Jupyter avec Gmail API", "ton_mail@gmail.com")

E-mail envoyé ✅ ID: 197bd12ee2029ea8


## Erreurs fréquentes

| ❌ Erreur                                                                 | 💡 Solution                                                                                                                                                             |
|--------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `redirect_uri_mismatch`                                                 | Ajoute **`http://localhost:8083/`** dans la console Google Cloud : `APIs & Services` → `Identifiants` → `ID client OAuth 2.0` → URI de redirection autorisé            |
| `MismatchingStateError`                                                 | Ne pas déclencher l'auth depuis Streamlit. Utilise un **script Python séparé** (`authenticate_gmail.py`) avec `run_local_server()` pour générer le `token.pickle`     |
| `access_denied`                                                         | Ton appli est en mode test. **Ajoute ton compte Gmail** dans la liste des **utilisateurs testeurs** dans la console Google Cloud                                      |
| `OSError: [Errno 98] Address already in use`                            | Le port utilisé (ex. 8083) est déjà occupé. Essaye un autre port (ex. 8084) **et ajoute aussi ce port dans la liste des URI autorisés dans Google Cloud**             |
| Aucun e-mail reçu                                                       | - Vérifie que le **fichier `token.pickle`** a bien été généré  <br> - Que l’adresse email de destination est correcte  <br> - Regarde dans le **dossier Spam**         |
| `invalid_request: access_type 'offline*' is not valid`                 | Ne pas ajouter manuellement `access_type=offline*` dans l’URL. Laisser `InstalledAppFlow` gérer automatiquement la demande d’autorisation                             |
| `gio: URL: Operation not supported`                                     | Ignore cette erreur : **`gio`** essaie d’ouvrir le navigateur automatiquement. Utilise le lien dans le terminal pour autoriser manuellement                           |